In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

ImportError: DLL load failed while importing _cext: The specified module could not be found.

In [2]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits, on = 'title')

In [4]:
movies = movies[['movie_id', 'title', 'genres', 'keywords', 'cast', 'crew']]

In [5]:
import ast 

def convert(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

In [6]:
movies['genres'] = movies['genres'].apply(convert)

In [18]:
movies.head(1)

,movie_id,title,genres,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [8]:
movies['keywords'] = movies['keywords'].apply(convert)

In [10]:
def convert_cast(text):
    l = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
           l.append(i['name'])
        counter +=1
    return l

In [11]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [13]:
def fectch_director(text):
    l = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [14]:
movies['crew'] = movies['crew'].apply(fectch_director)

In [16]:
def remove_space(word):
    l = []
    for i in word:
            l.append(i.replace(" ", ""))
    return l

In [17]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [19]:
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [20]:
new_data = movies[['movie_id', 'title', 'tags']]

In [21]:
new_data.shape

(4809, 3)

In [26]:
new_data.head(1)

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...


In [23]:
new_data['tags'] = new_data['tags'].apply(lambda x: " ".join(x))

C:\Users\RULoaner\AppData\Local\Temp\ipykernel_14784\1366722170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags'] = new_data['tags'].apply(lambda x: " ".join(x))


In [25]:
new_data['tags'] = new_data['tags'].apply(lambda x:x.lower())

C:\Users\RULoaner\AppData\Local\Temp\ipykernel_14784\2492974546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags'] = new_data['tags'].apply(lambda x:x.lower())


In [27]:
import nltk
from nltk.stem import PorterStemmer

In [28]:
ps = PorterStemmer()

In [29]:
def steams(text): 
    l = []
    for i in text.split():
        l.append(ps.stem(i))

    return " ".join(l)

In [30]:
new_data['tags'] = new_data['tags'].apply(steams)

C:\Users\RULoaner\AppData\Local\Temp\ipykernel_14784\2412174396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags'] = new_data['tags'].apply(steams)


In [31]:
new_data.iloc[0]['tags']

'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words= 'english')

In [34]:
vector = cv.fit_transform(new_data['tags']).toarray()

In [36]:
vector.shape

(4809, 5000)

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
similary = cosine_similarity(vector)

In [40]:
similary.shape

(4809, 4809)

In [41]:
new_data[new_data['title'] == 'Spider-Man'].index[0]

np.int64(159)

In [49]:
def recommend(movie):
    index = new_data[new_data['title'] == movie].index[0]
    distances = sorted(list(enumerate(similary[index])), reverse=True , key = lambda x:x[1])
    for i in distances[1:6]:
        print(new_data.iloc[i[0]].title)

In [52]:
recommend('The Dark Knight')

The Dark Knight Rises
Batman Begins
Harsh Times
Batman
Batman Returns


In [55]:
import pickle 

pickle.dump(new_data, open('artificats/movie_list.pkl', 'wb'))
pickle.dump(similary, open('artificats/similarity.pkl', 'wb'))